### 构建神经网络

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

#### 读取数据
- 分别构建训练集和测试集（验证集）
- DataLoader 来迭代取数据

In [2]:
# 定义超参数
input_size = 28     # 图像尺寸28*28
num_classes = 10    # 标签种类
num_epochs = 3      # 训练迭代次数
batch_size = 64     # 64张/批次

# 训练集
train_dataset = datasets.MNIST(root='./data',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

# 测试集
test_dataset = datasets.MNIST(root='./data',
                              train=False,
                              transform=transforms.ToTensor())

# 构建 batch 数据
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=True)


#### 卷积网络模块构建
- 一般卷积层，ReLU层，池化层
- 卷积最后结果还是一个特征图，需要把图转换成向量才能做分类或者回归任务

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Sequential(         # 输入大小 (1, 28, 28)
            nn.Conv2d(in_channels=1,        # 灰度图
                      out_channels=16,      # 特征图个数
                      kernel_size=5,        # 卷积核大小
                      stride=1,             # 步长
                      padding=2             # 如果希望卷积后尺寸跟原来一样，
                                            # 需要设置 padding=(kernel_size - 1)/2 if stride=1
                      ),                    # 输出特征图为 (16, 28, 28)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),    # 池化 (2*2)，输出结果为 (16, 14, 14)
        )

        self.conv2 = nn.Sequential(         # 输入大小 (16, 14, 14)
            nn.Conv2d(in_channels=16,
                      out_channels=32,
                      kernel_size=5,
                      stride=1,
                      padding=2),           # 输出 (32, 14, 14)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)     # 输出 (32, 7, 7)
        )

        self.out = nn.Linear(
            in_features=32*7*7,
            out_features=10)                # 全连接层

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)           # flatten 操作，结果为 (batch_size, 32*7*7)
        output = self.out(x)
        return output


#### 准确率作为评估标准

In [4]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)

#### 训练网络模型

In [5]:
# 实例化
net = CNN()
# 损失函数
crossEntropy = nn.CrossEntropyLoss()
# 优化器
optimizer = optim.Adam(net.parameters(), lr=0.001)  # 定义优化器，普通随机梯度下降算法

# train
for epoch in range(num_epochs):
    # 当前 epoch 的结果保存下来
    train_rights = []

    for batch_idx, (data, target) in enumerate(train_loader):   # 针对容器中的每一批进行循环
        net.train()
        output = net(data)
        loss = crossEntropy(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)

        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []

            for (data, target) in test_loader:
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)

            # 准确率计算
            train_r = (
                sum([tup[0] for tup in train_rights]),
                sum([tup[1] for tup in train_rights])
            )
            val_r = (
                sum([tup[0] for tup in val_rights]),
                sum([tup[1] for tup in val_rights])
            )

            print('当前 epoch: {} [{}/{} ({:.0f}%)]\t损失: {:.6f}\t训练集准确率: {:.2f}%\t测试集准确率: {:.2f}%'.format(
                epoch, batch_idx*batch_size, len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data,
                100. * train_r[0].numpy() / train_r[1],
                100. * val_r[0].numpy() / val_r[1]
            ))


当前 epoch: 0 [0/60000 (0%)]	损失: 2.310593	训练集准确率: 6.25%	测试集准确率: 11.81%
当前 epoch: 0 [6400/60000 (11%)]	损失: 0.193468	训练集准确率: 75.08%	测试集准确率: 91.27%
当前 epoch: 0 [12800/60000 (21%)]	损失: 0.196862	训练集准确率: 83.62%	测试集准确率: 94.73%
当前 epoch: 0 [19200/60000 (32%)]	损失: 0.033648	训练集准确率: 87.31%	测试集准确率: 94.44%
当前 epoch: 0 [25600/60000 (43%)]	损失: 0.081400	训练集准确率: 89.48%	测试集准确率: 97.00%
当前 epoch: 0 [32000/60000 (53%)]	损失: 0.127010	训练集准确率: 90.92%	测试集准确率: 97.22%
当前 epoch: 0 [38400/60000 (64%)]	损失: 0.043472	训练集准确率: 91.85%	测试集准确率: 97.77%
当前 epoch: 0 [44800/60000 (75%)]	损失: 0.038948	训练集准确率: 92.63%	测试集准确率: 97.92%
当前 epoch: 0 [51200/60000 (85%)]	损失: 0.022165	训练集准确率: 93.28%	测试集准确率: 97.96%
当前 epoch: 0 [57600/60000 (96%)]	损失: 0.044342	训练集准确率: 93.80%	测试集准确率: 98.00%
当前 epoch: 1 [0/60000 (0%)]	损失: 0.296286	训练集准确率: 92.19%	测试集准确率: 98.21%
当前 epoch: 1 [6400/60000 (11%)]	损失: 0.089358	训练集准确率: 98.08%	测试集准确率: 98.24%
当前 epoch: 1 [12800/60000 (21%)]	损失: 0.024019	训练集准确率: 98.13%	测试集准确率: 98.38%
当前 epoch: 1 [19200/60000 (32%)]	损失: 0.